# Проверка и очистка данных

## Контекст
Анализ выполнен на реальных коммерческих данных продаж зоомагазина за 2025 год.
В связи с конфиденциальностью коммерческих данных исходные данные не публикуются.

## Цель этапа
- Проверить структуру данных
- Обнаружить пропуски и несогласованности
- Подготовить рабочий датасет для EDA

### Данные не публикуются. Ноутбук демонстрирует логику обработки.


## 1. Импорт библиотек 


In [1]:
# Работа с данными 
import numpy as np
import pandas as pd
import re

## 2. Загрузка данных

In [2]:
# Загружаем подготовленный Excel-файл
df_work = pd.read_excel('../data/sales_2025_work.xlsx').copy()

# Быстрый просмотр первых строк
# df_work.head()
# Вывод строк данных отключён: используется реальная коммерческая выгрузка

## 3. Переименование колонок

In [3]:
df_work.rename(columns={
    'Название': 'product_name',
    'Категория': 'category',
    'Кол-во': 'qty',
    'Ед. изм.': 'unit',
    'Доля себест., %': 'cost_share_pct',
    'Себест.': 'cost',
    'Сумма себест.': 'cost_total',
    'Сумма себест. (без НДС)': 'cost_total_no_vat',
    'Себест., НДС': 'cost_vat',
    'Наценка %': 'markup_pct',
    'Сумма наценки': 'markup_total',
    'Доля наценки': 'markup_share_pct',
    'Сумма продажи (без скидки)': 'sales_total_no_discount',
    'Скидка %': 'discount_pct',
    'Сумма скидки': 'discount_total',
    'Доля, %': 'sales_share_pct',
    'Маржа, %': 'margin_pct',
    'Ср. цена': 'avg_price',
    'Сумма': 'sales_total',
    'НДС': 'vat',
    'Сумма (без НДС)': 'sales_total_no_vat',
    'Возвратов': 'returns_qty',
    'Сумма себст. \nс уч. возвратов': 'cost_total_with_returns',
    'Сумма возвратов': 'returns_total',
    'Сумма с уч. возвратов': 'sales_total_with_returns',
    'Сумма наценки с уч. возвратов': 'markup_total_with_returns'
}, inplace=True)

## 4. Первичная проверка данных

In [4]:
# Общая информация о датасете 
# df_work.info()

In [5]:
# Удаление ненужных колонок
cols_to_drop = [
    'vat', # почти все значения пустые
    'sales_total_no_vat', # данные не были учтены в системе
    'returns_qty', # все значения пустые 
    'returns_total', # все значения пустые
    'markup_share_pct'
]

df_work.drop(columns=cols_to_drop, inplace=True)

In [6]:
# Статистика числовых колонок
# df_work.describe()
# Агрегированные статистики не выводятся по соображениям конфиденциальности

In [7]:
# Пропуски по колонкам
missing = df_work.isna().sum()
print("Пропуски по колонкам:\n", missing)

Пропуски по колонкам:
 product_name                   10
category                       11
qty                            12
unit                           10
cost_share_pct                108
cost                          106
cost_total                    106
cost_total_no_vat             106
cost_vat                      720
markup_pct                    106
markup_total                   11
sales_total_no_discount        11
discount_pct                 1970
discount_total               1970
sales_share_pct              1218
margin_pct                    107
avg_price                      12
sales_total                    12
cost_total_with_returns       113
sales_total_with_returns        7
markup_total_with_returns       6
dtype: int64


In [8]:
# Удаление строк без полезной информации 
df_work = df_work.dropna(subset=['product_name','qty','sales_total'])

In [9]:
# Замена NaN, означающего отсутсвие скидки на 0
df_work['discount_pct'] = df_work['discount_pct'].fillna(0)
df_work['discount_total'] = df_work['discount_total'].fillna(0)

In [10]:
# Unknown для единиц измерения 
df_work['unit'] = df_work['unit'].fillna('unknown')

In [11]:
# Заполнение пропусков медианой 
num_cols_fill = ['cost','cost_total','markup_pct','margin_pct','avg_price','cost_total_with_returns','sales_total_with_returns','markup_total_with_returns']
for col in num_cols_fill:
    df_work[col] = df_work[col].fillna(df_work[col].median())

## 5. Приведение типов данных

In [12]:
# Категориальные колонки
cat_cols = ['product_name', 'unit', 'category']
for col in cat_cols:
    if col in df_work.columns:
        df_work[col] = df_work[col].astype('category')

# Целочисленные колонки
df_work['qty'] = df_work['qty'].astype(int)

In [13]:
# Проверка типов данных
df_work.dtypes

product_name                 category
category                     category
qty                             int64
unit                         category
cost_share_pct                float64
cost                          float64
cost_total                    float64
cost_total_no_vat             float64
cost_vat                      float64
markup_pct                    float64
markup_total                  float64
sales_total_no_discount       float64
discount_pct                  float64
discount_total                float64
sales_share_pct               float64
margin_pct                    float64
avg_price                     float64
sales_total                   float64
cost_total_with_returns       float64
sales_total_with_returns      float64
markup_total_with_returns     float64
dtype: object

## 6. Сохранение финального датасета

In [14]:
df_work.to_csv(
    '../data/sales_2025_cleaned.csv',
    index=False
)

## 7. Вывод

In [15]:
# Размер финального датасета
# df_work.shape

In [16]:
# Проверка финальной структуры датасета
# df_work.info()

### ИТОГ 
- Данные приведены к согласованной и аналитически интерпретируемой структуре
- Зафиксированы первичные ограничения качества данных
- Сформирован рабочий датасет для EDA и валидации бизнес-метрик
